In [ ]:
from pyomo.environ import *
import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
sys.path.append('../../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set
from energiapy.utils.scale_utils import scale_list, scale_tuple

In [ ]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 52  # Number of execution scenarios                     (chi)

M = 1e4  # Big M

In [ ]:
def build_model(cap_factor):
    
    # Define temporal scales
    scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=M, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system')
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    com1_loc3_out = Resource(name='com1_loc3_out', label='Commodity 1 sent out from location 3')
    com1_loc4_out = Resource(name='com1_loc4_out', label='Commodity 1 sent out from location 4')
    com1_loc5_out = Resource(name='com1_loc5_out', label='Commodity 1 sent out from location 5')
    com1_loc6_out = Resource(name='com1_loc6_out', label='Commodity 1 sent out from location 6')
    com1_loc7_out = Resource(name='com1_loc7_out', label='Commodity 1 sent out from location 7')
    com1_loc8_out = Resource(name='com1_loc8_out', label='Commodity 1 sent out from location 8')
    com1_loc9_out = Resource(name='com1_loc9_out', label='Commodity 1 sent out from location 9')
    com1_loc10_out = Resource(name='com1_loc10_out', label='Commodity 1 sent out from location 10')
    com1_loc11_out = Resource(name='com1_loc11_out', label='Commodity 1 sent out from location 11')
    com1_loc12_out = Resource(name='com1_loc12_out', label='Commodity 1 sent out from location 12')
    com1_loc13_out = Resource(name='com1_loc13_out', label='Commodity 1 sent out from location 13')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 250
    
    # prod_max = {0: 0.25*com1_process_capacity, 1: 0.5*com1_process_capacity, 2: 0.75*com1_process_capacity, 3: 0.95*com1_process_capacity, 4: com1_process_capacity}
    # prod_min = {0: 0, 1: 0.25*com1_process_capacity, 2: 0.5*com1_process_capacity, 3: 0.75*com1_process_capacity, 4: 0.95*com1_process_capacity}
    # rate_max = {0:1.25/2, 1: 1/2, 2: 0.75/2, 3: 0.5/2, 4: 0.25/2}
    # mode_ramp = {(0,1): 5, (1,2): 5}
    
    com1_procure = Process(name='procure com1', prod_max=150, conversion={com1_pur: -1, com1_in: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Procure com1')
    com1_sell = Process(name='sell com1', prod_max=150, conversion={com1_out: -1, com1_sold: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Sell com1')
    
    vopex_t1 = 0.01
    vopex_t2_low = 0.02
    vopex_t2_mid = 0.05
    vopex_t2_high = 0.08
    vopex_t3_low = 0.02
    vopex_t3_high = 0.04
    vopex_t4 = 0.01
    
    # capex_t1 = 0.01
    # capex_t2_low = 0.02
    # capex_t2_mid = 0.05
    # capex_t2_high = 0.08
    # capex_t3_low = 0.02
    # capex_t3_high = 0.04
    # capex_t4 = 0.01
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Commodity 1 received from location 1')
    com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Commodity 1 received from location 2')
    com1_receive_loc3 = Process(name='com1_receive_loc3', prod_max=com1_process_capacity, conversion={com1_loc3_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Commodity 1 received from location 3')
    com1_receive_loc4 = Process(name='com1_receive_loc4', prod_max=com1_process_capacity, conversion={com1_loc4_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_high, prod_min=0.01, label='Commodity 1 received from location 4')
    com1_receive_loc5 = Process(name='com1_receive_loc5', prod_max=com1_process_capacity, conversion={com1_loc5_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_mid, prod_min=0.01, label='Commodity 1 received from location 5')
    com1_receive_loc6 = Process(name='com1_receive_loc6', prod_max=com1_process_capacity, conversion={com1_loc6_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_low, prod_min=0.01, label='Commodity 1 received from location 6')
    com1_receive_loc7 = Process(name='com1_receive_loc7', prod_max=com1_process_capacity, conversion={com1_loc7_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_mid, prod_min=0.01, label='Commodity 1 received from location 7')
    com1_receive_loc8 = Process(name='com1_receive_loc8', prod_max=com1_process_capacity, conversion={com1_loc8_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_high, prod_min=0.01, label='Commodity 1 received from location 8')
    com1_receive_loc9 = Process(name='com1_receive_loc9', prod_max=com1_process_capacity, conversion={com1_loc9_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t3_high, prod_min=0.01, label='Commodity 1 received from location 9')
    com1_receive_loc10 = Process(name='com1_receive_loc10', prod_max=com1_process_capacity, conversion={com1_loc10_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t3_low, prod_min=0.01, label='Commodity 1 received from location 10')
    com1_receive_loc11 = Process(name='com1_receive_loc11', prod_max=com1_process_capacity, conversion={com1_loc11_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t3_high, prod_min=0.01, label='Commodity 1 received from location 11')
    com1_receive_loc12 = Process(name='com1_receive_loc12', prod_max=com1_process_capacity, conversion={com1_loc12_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t4, prod_min=0.01, label='Commodity 1 received from location 12')
    com1_receive_loc13 = Process(name='com1_receive_loc13', prod_max=com1_process_capacity, conversion={com1_loc13_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t4, prod_min=0.01, label='Commodity 1 received from location 13')
    
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=30, vopex=0.01, prod_min=0.01, label='Process the commodity through the location', varying=[VaryingProcess.DETERMINISTIC_CAPACITY])
    
    com1_store = Process(name='com1_store', prod_max=com1_process_capacity, capex=0.1, vopex=0.01, storage_capex=10, store_min=0.01, store_max= 300, prod_min=0.01, label="Storage process", storage=com1_in, storage_cost=0.02)
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Send commodity one from location 1')
    com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Send commodity one from location 2')
    com1_loc3_send = Process(name='com1_loc3_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc3_out:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Send commodity one from location 3')
    com1_loc4_send = Process(name='com1_loc4_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc4_out:1}, capex=0.1, vopex=vopex_t2_high, prod_min=0.01, label='Send commodity one from location 4')
    com1_loc5_send = Process(name='com1_loc5_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc5_out:1}, capex=0.1, vopex=vopex_t2_mid, prod_min=0.01, label='Send commodity one from location 5')
    com1_loc6_send = Process(name='com1_loc6_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc6_out:1}, capex=0.1, vopex=vopex_t2_low, prod_min=0.01, label='Send commodity one from location 6')
    com1_loc7_send = Process(name='com1_loc7_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc7_out:1}, capex=0.1, vopex=vopex_t2_mid, prod_min=0.01, label='Send commodity one from location 7')
    com1_loc8_send = Process(name='com1_loc8_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc8_out:1}, capex=0.1, vopex=vopex_t2_high, prod_min=0.01, label='Send commodity one from location 8')
    com1_loc9_send = Process(name='com1_loc9_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc9_out:1}, capex=0.1, vopex=vopex_t3_high, prod_min=0.01, label='Send commodity one from location 9')
    com1_loc10_send = Process(name='com1_loc10_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc10_out:1}, capex=0.1, vopex=vopex_t3_low, prod_min=0.01, label='Send commodity one from location 10')
    com1_loc11_send = Process(name='com1_loc11_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc11_out:1}, capex=0.1, vopex=vopex_t3_high, prod_min=0.01, label='Send commodity one from location 11')
    com1_loc12_send = Process(name='com1_loc12_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc12_out:1}, capex=0.1, vopex=vopex_t4, prod_min=0.01, label='Send commodity one from location 12')
    com1_loc13_send = Process(name='com1_loc13_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc13_out:1}, capex=0.1, vopex=vopex_t4, prod_min=0.01, label='Send commodity one from location 13')
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_receive_loc4, com1_receive_loc5, com1_receive_loc6, com1_process, com1_store, com1_loc1_send}, label="Location 1", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc2 = Location(name='loc2', processes={com1_procure, com1_receive_loc5, com1_receive_loc6, com1_receive_loc7, com1_process, com1_store, com1_loc2_send}, label="Location 2", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, capacity_factor={com1_process: cap_factor[['com1_process']]})
    
    loc3 = Location(name='loc3', processes={com1_procure, com1_receive_loc6, com1_receive_loc7, com1_receive_loc8, com1_process, com1_store, com1_loc3_send}, label="Location 3", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc4 = Location(name='loc4', processes={com1_receive_loc1, com1_receive_loc9, com1_process, com1_store, com1_loc4_send}, label="Location 4", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc5 = Location(name='loc5', processes={com1_receive_loc1, com1_receive_loc2, com1_receive_loc9, com1_process, com1_store, com1_loc5_send}, label="Location 5", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc6 = Location(name='loc6', processes={com1_receive_loc1, com1_receive_loc2, com1_receive_loc3, com1_receive_loc9, com1_receive_loc10, com1_process, com1_store, com1_loc6_send}, label="Location 6", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, capacity_factor={com1_process: cap_factor[['com1_process']]})
    
    loc7 = Location(name='loc7', processes={com1_receive_loc2, com1_receive_loc3, com1_receive_loc10, com1_receive_loc11, com1_process, com1_store, com1_loc7_send}, label="Location 7", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc8 = Location(name='loc8', processes={com1_receive_loc3, com1_receive_loc11, com1_process, com1_store, com1_loc8_send}, label="Location 8", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc9 = Location(name='loc9', processes={com1_receive_loc4, com1_receive_loc5, com1_receive_loc6, com1_receive_loc12, com1_process, com1_store, com1_loc9_send}, label="Location 9", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc10 = Location(name='loc10', processes={com1_receive_loc6, com1_receive_loc7, com1_receive_loc12, com1_receive_loc13, com1_process, com1_store, com1_loc10_send}, label="Location 10", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, capacity_factor={com1_process: cap_factor[['com1_process']]})
    
    loc11 = Location(name='loc11', processes={com1_receive_loc7, com1_receive_loc8, com1_receive_loc13, com1_process, com1_store, com1_loc11_send}, label="Location 9", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc12 = Location(name='loc12', processes={com1_receive_loc9, com1_receive_loc10, com1_process, com1_store, com1_loc12_send, com1_sell}, label="Location 12", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc13 = Location(name='loc13', processes={com1_receive_loc10, com1_receive_loc11, com1_process, com1_store, com1_loc13_send, com1_sell}, label="Location 13", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    
    # ======================================================================================================================
    # Declare transport/trucks
    # ======================================================================================================================
    
    truck_large = 200
    truck_medium = 125
    truck_small = 75
    
    truck14 = Transport(name='truck14', resources={com1_loc1_out}, trans_max=truck_small, label='Truck from location 1 to 4', capex=0.5, vopex=0.05, trans_min=truck_small)
    truck41 = Transport(name='truck41', resources={com1_loc4_out}, trans_max=truck_small, label='Truck from location 4 to 1', capex=0.0001, vopex=0.05, trans_min=truck_small)
    
    truck15 = Transport(name='truck15', resources={com1_loc1_out}, trans_max=truck_medium, label='Truck from location 1 to 5', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck51 = Transport(name='truck51', resources={com1_loc5_out}, trans_max=truck_medium, label='Truck from location 5 to 1', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck16 = Transport(name='truck16', resources={com1_loc1_out}, trans_max=truck_medium, label='Truck from location 1 to 6', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck61 = Transport(name='truck61', resources={com1_loc6_out}, trans_max=truck_medium, label='Truck from location 6 to 1', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck25 = Transport(name='truck25', resources={com1_loc2_out}, trans_max=truck_medium, label='Truck from location 2 to 5', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck52 = Transport(name='truck52', resources={com1_loc5_out}, trans_max=truck_medium, label='Truck from location 5 to 2', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck26 = Transport(name='truck26', resources={com1_loc2_out}, trans_max=truck_medium, label='Truck from location 2 to 6', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck62 = Transport(name='truck62', resources={com1_loc6_out}, trans_max=truck_medium, label='Truck from location 6 to 2', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck27 = Transport(name='truck27', resources={com1_loc2_out}, trans_max=truck_medium, label='Truck from location 2 to 7', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck72 = Transport(name='truck72', resources={com1_loc7_out}, trans_max=truck_medium, label='Truck from location 7 to 2', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck36 = Transport(name='truck36', resources={com1_loc3_out}, trans_max=truck_medium, label='Truck from location 3 to 6', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck63 = Transport(name='truck63', resources={com1_loc6_out}, trans_max=truck_medium, label='Truck from location 6 to 3', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck37 = Transport(name='truck37', resources={com1_loc3_out}, trans_max=truck_medium, label='Truck from location 3 to 7', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck73 = Transport(name='truck73', resources={com1_loc7_out}, trans_max=truck_medium, label='Truck from location 7 to 3', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck38 = Transport(name='truck38', resources={com1_loc3_out}, trans_max=truck_small, label='Truck from location 3 to 8', capex=0.5, vopex=0.05, trans_min=truck_small)
    truck83 = Transport(name='truck83', resources={com1_loc8_out}, trans_max=truck_small, label='Truck from location 8 to 3', capex=0.0001, vopex=0.05, trans_min=truck_small)
    
    truck49 = Transport(name='truck49', resources={com1_loc4_out}, trans_max=truck_medium, label='Truck from location 4 to 9', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck94 = Transport(name='truck94', resources={com1_loc9_out}, trans_max=truck_medium, label='Truck from location 9 to 4', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck59 = Transport(name='truck59', resources={com1_loc5_out}, trans_max=truck_medium, label='Truck from location 5 to 9', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck95 = Transport(name='truck95', resources={com1_loc9_out}, trans_max=truck_medium, label='Truck from location 9 to 5', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck69 = Transport(name='truck69', resources={com1_loc6_out}, trans_max=truck_large, label='Truck from location 6 to 9', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck96 = Transport(name='truck96', resources={com1_loc9_out}, trans_max=truck_large, label='Truck from location 9 to 6', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck610 = Transport(name='truck610', resources={com1_loc6_out}, trans_max=truck_large, label='Truck from location 6 to 10', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck106 = Transport(name='truck106', resources={com1_loc10_out}, trans_max=truck_large, label='Truck from location 10 to 6', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck710 = Transport(name='truck710', resources={com1_loc7_out}, trans_max=truck_medium, label='Truck from location 7 to 10', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck107 = Transport(name='truck107', resources={com1_loc10_out}, trans_max=truck_medium, label='Truck from location 10 to 7', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck711 = Transport(name='truck711', resources={com1_loc7_out}, trans_max=truck_medium, label='Truck from location 7 to 11', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck117 = Transport(name='truck117', resources={com1_loc11_out}, trans_max=truck_medium, label='Truck from location 11 to 7', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck811 = Transport(name='truck811', resources={com1_loc8_out}, trans_max=truck_medium, label='Truck from location 8 to 11', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck118 = Transport(name='truck118', resources={com1_loc11_out}, trans_max=truck_medium, label='Truck from location 11 to 8', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck912 = Transport(name='truck912', resources={com1_loc9_out}, trans_max=truck_large, label='Truck from location 9 to 12', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck129 = Transport(name='truck129', resources={com1_loc12_out}, trans_max=truck_large, label='Truck from location 12 to 9', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck1012 = Transport(name='truck1012', resources={com1_loc10_out}, trans_max=truck_large, label='Truck from location 10 to 12', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck1210 = Transport(name='truck1210', resources={com1_loc12_out}, trans_max=truck_large, label='Truck from location 12 to 10', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck1013 = Transport(name='truck1013', resources={com1_loc10_out}, trans_max=truck_large, label='Truck from location 10 to 13', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck1310 = Transport(name='truck1310', resources={com1_loc13_out}, trans_max=truck_large, label='Truck from location 13 to 10', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck1113 = Transport(name='truck1113', resources={com1_loc11_out}, trans_max=truck_large, label='Truck from location 11 to 13', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck1311 = Transport(name='truck1311', resources={com1_loc13_out}, trans_max=truck_large, label='Truck from location 13 to 11', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    
    # ======================================================================================================================
    # Declare network
    # ======================================================================================================================
    
    transport_matrix = [
        [[], [], [], [truck14], [truck15], [truck16], [], [], [], [], [], [], []], # source: Location 1
        [[], [], [], [], [truck25], [truck26], [truck27], [], [], [], [], [], []], # source: Location 2
        [[], [], [], [], [], [truck36], [truck37], [truck38], [], [], [], [], []], # source: Location 3
        [[truck41], [], [], [], [], [], [], [], [truck49], [], [], [], []], # source: Location 4
        [[truck51], [truck52], [], [], [], [], [], [], [truck59], [], [], [], []], # source: Location 5
        [[truck61], [truck62], [truck63], [], [], [], [], [], [truck69], [truck610], [], [], []], # source: Location 6
        [[], [truck72], [truck73], [], [], [], [], [], [], [truck710], [truck711], [], []], # source: Location 7
        [[], [], [truck83], [], [], [], [], [], [], [], [truck811], [], []], # source: Location 8
        [[], [], [], [truck94], [truck95], [truck96], [], [], [], [], [], [truck912], []], # source: Location 9
        [[], [], [], [], [], [truck106], [truck107], [], [], [], [], [truck1012], [truck1013]], # source: Location 10
        [[], [], [], [], [], [], [truck117], [truck118], [], [], [], [], [truck1113]], # source: Location 11
        [[], [], [], [], [], [], [], [], [truck129], [truck1210], [], [], []], # source: Location 12
        [[], [], [], [], [], [], [], [], [], [truck1310], [truck1311], [], []] # source: Location 13
    ]
    
    distance_matrix = [
        [0, M, M, 93, 134, 177, M, M, M, M, M, M, M],
        [M, 0, M, M, 120, 163, 106, M, M, M, M, M, M],
        [M, M, 0, M, M, 166, 127, 75, M, M, M, M, M],
        [93, M, M, 0, M, M, M, M, 146, M, M, M, M],
        [134, 120, M, M, 0, M, M, M, 163, M, M, M, M],
        [177, 163, 166, M, M, 0, M, M, 166, 251, M, M, M],
        [M, 106, 127, M, M, M, 0, M, M, 274, 101, M, M],
        [M, M, 75, M, M, M, M, 0, M, M, 139, M, M],
        [M, M, M, 146, 163, 166, M, M, 0, M, M, 234, M],
        [M, M, M, M, M, 251, 274, M, M, 0, M, 296, 294],
        [M, M, M, M, M, M, 101, 139, M, M, 0, M, 219],
        [M, M, M, M, M, M, M, M, 234, 296, M, 0, M],
        [M, M, M, M, M, M, M, M, M, 294, 219, M, 0]
    ]

    locset = [loc1, loc2, loc3, loc4, loc5, loc6, loc7, loc8, loc9, loc10, loc11, loc12, loc13]
    
    sources = locset
    sinks = locset
    
    network = Network(name='Network', scales= scales, source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix, distance_matrix=distance_matrix)
    
    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = {loc12: 250, loc13: 250}
    demand_penalty = {loc12: 50, loc13: 50}
    
    demand_dict = {i: {com1_sold: daily_demand[i]} if i in daily_demand else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty[i]} if i in daily_demand else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name='scenario', scales=scales, scheduling_scale_level=2, network_scale_level=0, purchase_scale_level=2, availability_scale_level=1, demand_scale_level=2, capacity_scale_level=1, network=network, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Stochastic scenario with Multiple Locations')
    
    # ======================================================================================================================
    # Declare problem
    # ======================================================================================================================

    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.TRANSPORT, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)

    scale_iter = scale_tuple(instance=problem_mincost, scale_levels=scenario.network_scale_level+1)
    capex_process= sum(problem_mincost.Capex_network[scale_] for scale_ in scale_iter)
    cost_trans_capex = sum(problem_mincost.Capex_transport_network[scale_] for scale_ in scale_iter)

    problem_mincost.first_stage_cost  = capex_process + cost_trans_capex
    
    return scenario, problem_mincost

# Scenario Information

In [ ]:
all_scenario_names = ["good", "average", "bad"]
ns_dict = {'good': 56, "average": 26, "bad": 0}
scenario_probabilities = {'good': 0.2, 'average': 0.6, 'bad': 0.2} 

# Solver Options

In [ ]:
solver_options = {
    'MIPGap': 0.005,
    'TimeLimit': 60 * 15,
    'Heuristics': 0.20,
    'NodefileStart': 0.5,
    'Threads': 20
}

In [ ]:
exCost_PI = 0
results_PI =  dict()
model_PI = dict()

for scenario_name in all_scenario_names:
    cap_factor_PI = pandas.DataFrame(data={'com1_process': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen_PI, model_PI[scenario_name] = build_model(cap_factor=cap_factor_PI)
    results_PI[scenario_name] = solve(scenario=scen_PI, instance=model_PI[scenario_name], solver='gurobi', name=scenario_name, solver_options=solver_options) 
    
    print('######################## Finished solving '+scenario_name+' ########################')
    
    exCost_PI += value(model_PI[scenario_name].objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

In [ ]:
exCost_PI

# Expected Value of Perfect Information

In [ ]:
def scenario_creator(scenario_name):  
    cap_factor_scen = pandas.DataFrame(data={'com1_process': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen, model = build_model(cap_factor=cap_factor_scen)
    sputils.attach_root_node(model, model.first_stage_cost, [model.X_P, model.Cap_P, model.X_S, model.Cap_S, model.X_F, model.Cap_F])
    model._mpisppy_probability = scenario_probabilities[scenario_name]
    return model

In [ ]:
from mpisppy.opt.ef import ExtensiveForm
options = {"solver": "gurobi"}
ef_UI = ExtensiveForm(options, all_scenario_names, scenario_creator)
results = ef_UI.solve_extensive_form(solver_options=solver_options)

In [ ]:
exCost_UI = ef_UI.get_objective_value()
print(f"{exCost_UI:.4f}")

In [ ]:
EVPI = exCost_UI - exCost_PI
EVPI